In [1]:
import itertools
import sys
import os
sys.path.append("../") # go to parent dir

import jax
import jax.random as jr
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import numpy as np
from scipy.stats import rankdata
import scipy.stats as ss
import seaborn as sns
from sklearn.model_selection import KFold

# from data.create_sim_data import *
import data.template_causl_simulations as causl_py
from data.run_all_simulations import plot_simulation_results
import data.hyperparam_and_bootstrapping as hb
from frugal_flows.causal_flows import independent_continuous_marginal_flow, get_independent_quantiles, train_frugal_flow
from frugal_flows.bijections import UnivariateNormalCDF

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage

# Activate automatic conversion of rpy2 objects to pandas objects
pandas2ri.activate()

# Import the R library causl
try:
    causl = importr('causl')
except Exception as e:
    package_names = ('causl')
    utils.install_packages(StrVector(package_names))

jax.config.update("jax_enable_x64", True)

hyperparams_dict = {
    'learning_rate': 5e-3,
    'RQS_knots': 8,
    'flow_layers': 5,
    'nn_width': 50,
    'nn_depth': 4,    
    'max_patience': 50,
    'max_epochs': 10000
}

NUM_ITER = 10
TRUE_PARAMS = {'ate': 1, 'const': 1, 'scale': 1}
CAUSAL_PARAMS = [1, 1]

## Generate Data

In [7]:
data = causl_py.generate_gaussian_samples(N=1000, causal_params=[1,1], seed=0)
Z_cont = data.get('Z_cont')
X = data.get('X')
Y = data.get('Y')

In [1]:
import jax
jax.devices()

[CpuDevice(id=0)]

In [18]:
# Define the hyperparameter ranges
param_grid = {
    'RQS_knots': [4, 6, 8],
    'flow_layers': [4, 6, 8],
    'nn_width': [20, 40, 60],
    'nn_depth': [4, 6, 8],
    'learning_rate': [3e-3, 5e-3],
    'batch_size': [1000],
    'max_patience': [50],
    'max_epochs': [10000]
}

param_combinations = hb.generate_param_combinations(param_grid)

In [19]:
hyperparam_fits = hb.gaussian_outcome_hyperparameter_search(
    X, Y, Z_disc=None, Z_cont=Z_cont, param_combinations=param_combinations, seed=0
)

  0%|                                                                                                                                                                                     | 0/10000 [00:00<?, ?it/s]2024-05-15 14:27:48.066138: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 2m31.406121s

********************************
[Compiling module jit___call__] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
  1%|█▎                                                                                                                       | 112/10000 [00:08<12:29, 13.20it/s, train=2.3493018463735926, val=2.3602388347776877]


KeyboardInterrupt: 

In [17]:
hyperparam_fits.sort_values('min_loss').head(20)

,RQS_knots,flow_layers,nn_width,nn_depth,learning_rate,batch_size,max_patience,max_epochs,ate,const,scale,min_loss
0,4,2,60,8,0.003,1000,50,10000,0.7150669160365015,1.374007716934389,1.4217528496579643,-1.3260963361708087
0,4,2,35,2,0.003,1000,50,10000,0.8266724675551651,1.5860350125175438,2.29718594724751,-1.3028112910257834
0,4,2,25,4,0.003,1000,50,10000,1.0179928860883678,1.665140150428186,2.572571572273175,-0.6892003449425886
0,4,2,45,6,0.003,1000,50,10000,0.8571017907300323,1.7412041588663232,2.4900306210544714,-0.6708703916855526
0,6,2,50,8,0.003,1000,50,10000,0.9446084566625047,1.5971757854820765,2.3495064545679925,-0.5243316967847134
0,6,2,25,4,0.003,1000,50,10000,1.4900972867872,1.9371309025109238,2.862495278420609,-0.5242080519884562
0,4,2,50,4,0.003,1000,50,10000,1.035237754176492,1.684316556302863,2.4932209816214552,-0.5143605829525498
0,8,2,25,4,0.003,1000,50,10000,1.736149392203217,1.9078992491636033,2.872326192508355,-0.4606091843220257
0,4,2,25,2,0.003,1000,50,10000,2.1145454335751657,2.2542805387658795,3.0904339231313105,-0.3695486533062358
0,4,2,45,8,0.003,1000,50,10000,1.7178410146193837,1.9942493167948057,2.925035631203214,-0.3574400345667037
